In [20]:
from datasets import load_dataset
import re
import pandas as pd
from transformers import pipeline


dataset = load_dataset("corbt/enron-emails")
classifier = pipeline('zero-shot-classification', model='roberta-large-mnli')
emails = [dataset['train'][i]['body'] for i in range(2000)]

def clean_email(text):
    text = re.sub(r'(?s)^.*?\n\n', '', text)
    text = re.sub(r'(?i)(--|__|Thanks,|Regards,|Sent from.*|Best,).*$','', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[\w\.-]+@[\w\.-]+', '', text)
    text = re.sub(r'(?s)(On\s.*wrote:|From:.*\nTo:.*\nSubject:.*\n).*', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\n{2,}', '\n', text)
    return text.strip()

labels = [
    "Course Registration",
    "Documents & Certificates",
    "Payment & Fees",
    "Scheduling & Attendance",
    "General Inquiry"
]

def assign_label(text):
    if not text:
        return "General Inquiry"
    text_lower = text.lower()
    obj = classifier(text, labels)
    return obj["labels"][0]

def clean_and_label(email):
    cleaned = clean_email(email)
    label = assign_label(cleaned)
    print("completed")
    return cleaned, label

processed = [clean_and_label(email) for email in emails]

df = pd.DataFrame(processed, columns=["text", "label"])
df.to_csv("enron_emails_labeled.csv", index=False)

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


completed
completed
completed


KeyboardInterrupt: 

In [21]:
from datasets import load_dataset
import pandas as pd

# Load first 2000 emails from Hugging Face
dataset = load_dataset("corbt/enron-emails", split="train[:2000]")

# Convert to DataFrame
df = pd.DataFrame(dataset)

# Save locally
df.to_csv("enron_emails_raw.csv", index=False)  # OR use .json if preferred